In [1]:
# Load environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAI

from langchain.tools import tool
from langchain import hub
from langchain.prompts import load_prompt

from langchain.agents import AgentExecutor, create_react_agent

In [3]:
# Switch to use openAI
#llm = OpenAI(temperature=0, verbose=True, model="gpt-3.5-turbo-instruct") # defautl model : "gpt-3.5-turbo-instruct"
llm = Ollama(model="mistral")

In [4]:
# custom simple tool
@tool
def square(x: int) -> int:
    """
    squares a number
    """
    return int(x)**2

In [5]:
# tool features
print(square.name)
print(square.description)
print(square.args)

square
square(x: int) -> int - squares a number
{'x': {'title': 'X', 'type': 'integer'}}


In [7]:
# pull prompt from the hub and saves it 
obj = hub.pull("hwchase17/react")
obj.save("prompt.json")

In [8]:
# inititalize prompt and tools for custom agent
prompt = load_prompt('prompt.json')
tools = [
    square,
    call_API
    ]

In [9]:
# the prompt template
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
agent_custom = create_react_agent(
     llm,
     tools,
    prompt
    )

agent_executor = AgentExecutor(agent=agent_custom, tools=tools, verbose=True)

In [11]:
agent_executor.invoke(input= {'input':"What is the square of 5 "})



> Entering new AgentExecutor chain...
 I need to calculate the square of a number, so I will use the "square" tool.
Action: square
Action Input: 525 The observation tells me that the result of squaring 5 is 25.
Final Answer: The square of 5 is 25.

> Finished chain.


{'input': 'What is the square of 5 ', 'output': 'The square of 5 is 25.'}

In [17]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs
from langchain_openai import OpenAI

#llm = OpenAI(temperature=0, verbose=True, model="gpt-3.5-turbo-instruct")
llm = Ollama(model="mistral")
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)



> Entering new APIChain chain...
 Based on the user question and the given API documentation, I would suggest the following API URL to get the current temperature in Fahrenheit for Munich, Germany:

`https://api.open-meteo.com/v1/forecast?latitude=48.1352&longitude=11.5765&temperature_unit=fahrenheit&current_weather=true`

This API call requests the current weather data for the specified geographical coordinate (Munich's coordinates) with temperature units set to Fahrenheit and includes the current weather conditions in the JSON output.


ValueError: Based on the user question and the given API documentation, I would suggest the following API URL to get the current temperature in Fahrenheit for Munich, Germany:

`https://api.open-meteo.com/v1/forecast?latitude=48.1352&longitude=11.5765&temperature_unit=fahrenheit&current_weather=true`

This API call requests the current weather data for the specified geographical coordinate (Munich's coordinates) with temperature units set to Fahrenheit and includes the current weather conditions in the JSON output. is not in the allowed domains: ['https://api.open-meteo.com/']